<a href="https://colab.research.google.com/github/A-l-E-v/ML-Engineer/blob/main/Lesson_3_6_(Compositions).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
sns.set_theme(style="darkgrid")

## 1. Использование FacetGrid

In [ ]:
data_auto = sns.load_dataset('mpg')

In [ ]:
data_auto.head()

In [ ]:
# Создание объекта
fg = sns.FacetGrid(data_auto, col="origin")
plt.show()

In [ ]:
# Теперь можно сопоставить каждому графику диаграмму, которую мы хотим использовать
fg = sns.FacetGrid(data_auto, col="origin")
fg.map(sns.histplot, "mpg")
plt.show()

In [ ]:
# Чтобы сделать график зависимости, нужно передать несколько параметров в map
fg = sns.FacetGrid(data_auto, col="origin", hue="cylinders")
fg.map(sns.scatterplot, "mpg", "horsepower", alpha=.5)
fg.add_legend() # обратите внимание, что в данном случае легенду надо добавить самостоятельно!
plt.show()

In [ ]:
# можно контролировать внешний вид путем передачи параметров в FacetGrid,
# а параметры самой функции можно передавать в map

fg = sns.FacetGrid(data_auto, row="model_year", col="origin", margin_titles=True)
fg.map(sns.stripplot, "cylinders", "mpg", order=[3, 4, 5, 6, 8], jitter=.1)
plt.show()

In [ ]:
# Можно задавать рамер каждого графика в композиции
fg = sns.FacetGrid(data_auto, col="cylinders", height=5, aspect=.5)
fg.map(sns.barplot, "origin", "mpg", order=["usa", "europe", "japan"])
plt.show()

In [ ]:
# Работает также и со строками
fg = sns.FacetGrid(data_auto, row="origin", row_order=["usa", "europe", "japan"], aspect=4)
fg.map(sns.kdeplot, "mpg")
plt.show()

In [ ]:
# можно подбирать цвета с помощью palette
fg = sns.FacetGrid(data_auto, col="origin", hue="cylinders", palette='Oranges')
fg.map(sns.scatterplot, "mpg", "horsepower", alpha=.5)
fg.add_legend()
plt.show()

In [ ]:
# можно ограничить число столбцов c помощью wrap, причем тогда нельзя использовать параметр row
fg = sns.FacetGrid(data_auto, col="cylinders", col_wrap=3)
fg.map(sns.pointplot, "model_year", "mpg", color="g", order=data_auto['model_year'].unique(),
       errorbar=None, native_scale=True)
plt.show()

In [ ]:
# Кастомизация графика путём обращения к методам FacetGrid

with sns.axes_style("white"):
    fg = sns.FacetGrid(data_auto, col="origin")
fg.map(sns.histplot, "mpg")
fg.set_axis_labels("Fuel consumption (meter per gallon)", "Number of cars", size=10)
fg.set(xticks=np.arange(10, 50, 5), yticks=[0, 15, 30, 45])
plt.show()

In [ ]:
# ещё большая кастомиpация с помощью обращения непосредственно к объектам Axes и Figure класса FacetGrid

with sns.axes_style("white"):
    fg = sns.FacetGrid(data_auto, col="origin")
fg.map(sns.histplot, "mpg")
fg.set_axis_labels("Fuel consumption (meter per gallon)", "Number of cars", size=10)
fg.set(xticks=np.arange(10, 50, 5), yticks=[0, 15, 30, 45])

fg.fig.suptitle('Fuel consumption distribution by origin', fontsize=15, x=0.55)
fg.fig.subplots_adjust(wspace=.02, hspace=.02, top=0.8)

for ax in fg.axes_dict.values():
    ax.grid(color='black', linestyle=':')

plt.show()

## *2. Использование собственных функций в качестве параметра метода FacetGrid.map()*

Чтобы использовать собственную функцию в качестве параметра, она должна удовлетворять трём условиям:

1. Она должна выполнять построения в активные в текущем контексте объекты Axes. В случае неявного стиля программирования, достаточно использовать функции из модуля matplotlib.pyplot, которые выберут нужные объекты в теневом режиме. В случае, если используется явный стиль, можно получить необходимые объекты с помощью функции matplotlib.pyplot.gca(), а далее использовать полученный объект Axes и его методы

2. Она должна принимать сами данные, переданные до этого в качестве параметра для создания объекта FacetGrid. Впоследствии метод map использует созданную функцию, чтобы визуализировать данные на каждом из подграфиков

3. Функция должна принимать и уметь работать с параметрами *color* и *label*, которые контролируют цветовую гамму линий/точек и подписи подграфиков соответственно. **Проще всего реализовать такую функциональность, передав** *****kwargs*** **в качестве последнего параметра функции. Эта заглушка отвечает в python за сохранение всех параметров после именованных.**

In [ ]:
from scipy import stats

In [ ]:
# строим квантильный график
def quantile_plot(data, **kwargs):
    ax = plt.gca()
    quantiles, xr = stats.probplot(data, fit=False)
    ax.scatter(xr, quantiles, **kwargs)

fg = sns.FacetGrid(data_auto, col="origin", aspect=0.8, height=6)
fg.map(quantile_plot, "mpg")
fg.fig.suptitle('Fuel consumption distribution by origin, probability plots', fontsize=20, x=0.55)
fg.fig.subplots_adjust(top=0.85)
plt.show()

In [ ]:
# строим Q-Q график
def qq_plot(data_x, data_y, diagonal=True, **kwargs):
    ax = plt.gca()
    _, xr = stats.probplot(data_x, fit=False)
    _, yr = stats.probplot(data_y, fit=False)
    ax.scatter(xr, yr, **kwargs)
    if diagonal:
        ax.plot([np.nanmin(xr), np.nanmax(xr)], [np.nanmin(yr), np.nanmax(yr)], ls='--', lw=3, color='g')

fg = sns.FacetGrid(data_auto, col="origin", aspect=0.8, height=6, sharex=False, sharey=False)
fg.map(qq_plot, "mpg", "horsepower")
fg.fig.suptitle('Fuel consumption distribution by origin, Q-Q plots', fontsize=20, x=0.55)
fg.fig.subplots_adjust(top=0.85)
plt.show()

In [ ]:
# строим Q-Q график
def qq_plot(data_x, data_y, diagonal=True, **kwargs):
    ax = plt.gca()
    _, xr = stats.probplot(data_x, fit=False)
    _, yr = stats.probplot(data_y, fit=False)
    ax.scatter(xr, yr, **kwargs)
    if diagonal:
        ax.plot([np.nanmin(xr), np.nanmax(xr)], [np.nanmin(yr), np.nanmax(yr)], ls='--', lw=3, color='g')

fg = sns.FacetGrid(data_auto, col="origin", aspect=0.8, height=6, sharex=False, sharey=False)
fg.map(qq_plot, "mpg", "acceleration")
fg.fig.suptitle('Fuel consumption distribution by origin, Q-Q plots', fontsize=20, x=0.55)
fg.fig.subplots_adjust(top=0.85)
plt.show()

In [ ]:
# color тоже работает

fg = sns.FacetGrid(data_auto, col="origin", hue='cylinders', aspect=0.8, height=6, sharex=False, sharey=False)
fg.map(qq_plot, "mpg", "acceleration")
fg.fig.suptitle('Fuel consumption distribution by origin, Q-Q plots', fontsize=20, x=0.55)
fg.fig.subplots_adjust(top=0.85)
fg.add_legend()
plt.show()

In [ ]:
# при этом можно не пользоваться только **kwargs, а кастомизировать параметры по-своему

def hexbin(x, y, color, gridsize, **kwargs):
    cmap = sns.light_palette(color, as_cmap=True)
    plt.hexbin(x, y, gridsize=gridsize, cmap=cmap, **kwargs)

with sns.axes_style("dark"):
    g = sns.FacetGrid(data_auto, hue="origin", col="origin", height=6, sharex=False, sharey=False)
g.map(hexbin, "mpg", "horsepower", gridsize=15)
plt.show()

## 3. Использование PairGrid()

In [ ]:
# начало работы

pg = sns.PairGrid(data_auto)
pg.map(sns.scatterplot)
pg.fig.suptitle('Pairwise correlations for automobile data', fontsize=20, x=0.5)
pg.fig.subplots_adjust(top=0.95)
plt.show()

In [ ]:
# диагональные элементы можно изменить, при этом шкала оси y уже не будет верна для диаг. элементов

pg = sns.PairGrid(data_auto)
pg.map_diag(sns.histplot)
pg.map_offdiag(sns.scatterplot)
pg.fig.suptitle('Pairwise correlations for automobile data', fontsize=20, x=0.5)
pg.fig.subplots_adjust(top=0.95)
plt.show()

In [ ]:
# добавляем цветовое различие

pg = sns.PairGrid(data_auto, hue="origin")
pg.map_diag(sns.histplot)
pg.map_offdiag(sns.scatterplot)
pg.fig.suptitle('Pairwise correlations for automobile data (divided by origin)', fontsize=20, x=0.5)
pg.fig.subplots_adjust(top=0.95)
pg.add_legend()
plt.show()

In [ ]:
# выделяем только те величины что нам нужны

pg = sns.PairGrid(data_auto, vars=["mpg", "cylinders", "horsepower", "acceleration"], hue="origin")
pg.map(sns.scatterplot)
pg.fig.suptitle('Pairwise correlations for automobile data (short)', fontsize=20, x=0.5)
pg.fig.subplots_adjust(top=0.95)
pg.add_legend()
plt.show()

In [ ]:
# можно кастомизировать то, что под и над диагональю, отдельно

pg = sns.PairGrid(data_auto)
pg.map_upper(sns.scatterplot)
pg.map_lower(sns.kdeplot)
pg.map_diag(sns.histplot)
pg.fig.suptitle('Pairwise correlations for automobile data (customized)', fontsize=20, x=0.5)
pg.fig.subplots_adjust(top=0.95)
plt.show()

In [ ]:
# можно делать не только квадратную матрицу, её можно кастомизировать

pg = sns.PairGrid(data_auto, y_vars=["mpg"], x_vars=["acceleration", "cylinders"], height=6)
pg.map(sns.regplot, color="purple")
pg.axes[0][0].set_xticks(np.arange(7, 25.1, 2))
pg.fig.suptitle('Pairwise correlations for automobile data (mpg vs. acceleration/cylinders)',
                fontsize=20, x=0.5)
pg.fig.subplots_adjust(top=0.93)
plt.show()

In [ ]:
# можно кастомизировать внешний вид

pg = sns.PairGrid(data_auto, hue="cylinders", palette="Wistia")
pg.map(plt.scatter, s=25, marker='s')
pg.add_legend()
pg.fig.suptitle('Pairwise correlations for automobile data', fontsize=20, x=0.5)
pg.fig.subplots_adjust(top=0.95)
plt.show()

In [ ]:
# Всё же, для быстрой визуализации можно использовать функцию pairplot

fig = sns.pairplot(data_auto, hue="origin", palette="Set2", diag_kind="kde", height=3)
fig.fig.suptitle('Pairwise correlations for automobile data (designed)', fontsize=20, x=0.5)
fig.fig.subplots_adjust(top=0.95)
plt.show()